### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
#import setup
import pandas as pd

# Files to examine
schooldataload = "Resources/schools_complete.csv"
studentdataload = "Resources/students_complete.csv"

# Data Frames
schoolDF = pd.read_csv(schooldataload)
studentDF = pd.read_csv(studentdataload)

# Combine the data into a single dataset.  
completeDataDF = pd.merge(studentDF, schoolDF, how="left", on=["school_name", "school_name"])
completeDataDF.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#find summary values
schoolTotal = len(completeDataDF["school_name"].unique())
studentTotal = len(completeDataDF)
budgetTotal = sum(completeDataDF["budget"].unique())
averageMath = completeDataDF["math_score"].mean()
averageReading = completeDataDF["reading_score"].mean()
mathPassing = len(completeDataDF.loc[completeDataDF["math_score"] > 69])/studentTotal
readingPassing = len(completeDataDF.loc[completeDataDF["reading_score"] > 69])/studentTotal
overallPassing = len(completeDataDF.loc[(completeDataDF["math_score"] > 69) & (completeDataDF["reading_score"] > 69)])/studentTotal

#create summary dataframe with headers
summaryDF = pd.DataFrame({"Total Schools": [schoolTotal], "Total Students": [studentTotal], "Total Budget": [budgetTotal], 
                         "Math Score Average": [averageMath], "Reading Score Average": [averageReading], 
                         "Math Passing Percent": [mathPassing], "Reading Passing Percent": [readingPassing], 
                         "Overall Passing Percent": [overallPassing]})
#format columns
summaryDF["Total Students"] = summaryDF["Total Students"].map("{:,}".format)
summaryDF["Total Budget"] = summaryDF["Total Budget"].map("${:,.2f}".format)
summaryDF["Math Score Average"] = summaryDF["Math Score Average"].map("{:,.3f}".format)
summaryDF["Reading Score Average"] = summaryDF["Reading Score Average"].map("{:,.3f}".format)
summaryDF["Math Passing Percent"] = summaryDF["Math Passing Percent"].map("{:.2%}".format)
summaryDF["Reading Passing Percent"] = summaryDF["Reading Passing Percent"].map("{:.2%}".format)
summaryDF["Overall Passing Percent"] = summaryDF["Overall Passing Percent"].map("{:.2%}".format)

#display results
summaryDF.style.hide_index()

Total Schools,Total Students,Total Budget,Math Score Average,Reading Score Average,Math Passing Percent,Reading Passing Percent,Overall Passing Percent
15,"39,170","$24,649,428.00",78.985,81.878,74.98%,85.81%,65.17%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [3]:
schoolNameOverview = completeDataDF.groupby("school_name")
schoolName = schoolNameOverview.count().index
#budget, student count, passing grades, unique names
schoolType = schoolNameOverview["type"].unique()
totalStudent = schoolNameOverview["student_name"].count()
totalSchoolBudget = schoolNameOverview["budget"].unique()
budgetPerStudent = (schoolNameOverview["budget"].unique() / schoolNameOverview["student_name"].count())
averageMathScore = schoolNameOverview["math_score"].mean()
averageReadingScore = schoolNameOverview["reading_score"].mean()
#establish overall passing groups based on passing grades
mathPass = completeDataDF.loc[completeDataDF["math_score"] > 69, :]
mathPassOverview = mathPass.groupby("school_name")
mathPassing = (mathPassOverview["student_name"].count() / schoolNameOverview["student_name"].count())
readingPass = completeDataDF.loc[completeDataDF["reading_score"] > 69, :]
readingPassOverview = readingPass.groupby("school_name")
readingPassing = (readingPassOverview["student_name"].count() / schoolNameOverview["student_name"].count())
overallPass = completeDataDF.loc[((completeDataDF["math_score"] > 69) & (completeDataDF["reading_score"] > 69)), :]
overallPassOverview = overallPass.groupby("school_name")
overallPassing = (overallPassOverview["student_name"].count() / schoolNameOverview["student_name"].count())

#add to school dataframe
schoolGroupDF = pd.DataFrame({"School Name": schoolName, "School Type": schoolType.str[0], "Total Students": totalStudent, 
                             "Total Budget": totalSchoolBudget.str[0], "Budget Per Student": budgetPerStudent.str[0], 
                                "Average Math Score": averageMathScore, "Average Reading Score": averageReadingScore, 
                              "Math Passing Percent": mathPassing, "Reading Passing Percent": readingPassing, 
                              "Overall Passing Percent": overallPassing
                             })

#clean
schoolGroupDF = schoolGroupDF.reset_index(drop=True)

#format
schoolGroupDF["Total Students"] = schoolGroupDF["Total Students"].map("{:,}".format)
schoolGroupDF["Total Budget"] = schoolGroupDF["Total Budget"].map("${:,.2f}".format)
schoolGroupDF["Budget Per Student"] = schoolGroupDF["Budget Per Student"].map("${:,.2f}".format)
schoolGroupDF["Average Math Score"] = schoolGroupDF["Average Math Score"].map("{:,.3f}".format)
schoolGroupDF["Average Reading Score"] = schoolGroupDF["Average Reading Score"].map("{:,.3f}".format)
schoolGroupDF["Math Passing Percent"] = schoolGroupDF["Math Passing Percent"].map("{:.2%}".format)
schoolGroupDF["Reading Passing Percent"] = schoolGroupDF["Reading Passing Percent"].map("{:.2%}".format)
schoolGroupDF["Overall Passing Percent"] = schoolGroupDF["Overall Passing Percent"].map("{:.2%}".format)

schoolGroupDF.style.hide_index()

School Name,School Type,Total Students,Total Budget,Budget Per Student,Average Math Score,Average Reading Score,Math Passing Percent,Reading Passing Percent,Overall Passing Percent
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048,81.034,66.68%,81.93%,54.64%
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.062,83.976,94.13%,97.04%,91.33%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.712,81.158,65.99%,80.74%,53.20%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.103,80.746,68.31%,79.30%,54.29%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351,83.817,93.39%,97.14%,90.60%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.290,80.934,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087.00",$581.00,83.803,83.815,92.51%,96.25%,89.23%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629,81.183,65.68%,81.32%,53.51%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072,80.966,66.06%,81.22%,53.54%
Pena High School,Charter,962,"$585,858.00",$609.00,83.840,84.045,94.59%,95.95%,90.54%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [4]:
#create performing dataframe
topPerfDF = schoolGroupDF.sort_values("Overall Passing Percent", ascending=False)

#Top 5
topPerfDF.head(5).style.hide_index()

School Name,School Type,Total Students,Total Budget,Budget Per Student,Average Math Score,Average Reading Score,Math Passing Percent,Reading Passing Percent,Overall Passing Percent
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.062,83.976,94.13%,97.04%,91.33%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418,83.849,93.27%,97.31%,90.95%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351,83.817,93.39%,97.14%,90.60%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274,83.989,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858.00",$609.00,83.840,84.045,94.59%,95.95%,90.54%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [5]:
#create bottom performing DF
bottomPerfDF = schoolGroupDF.sort_values("Overall Passing Percent", ascending=True)

#Top 5
bottomPerfDF.head(5).style.hide_index()


School Name,School Type,Total Students,Total Budget,Budget Per Student,Average Math Score,Average Reading Score,Math Passing Percent,Reading Passing Percent,Overall Passing Percent
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.843,80.745,66.37%,80.22%,52.99%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.712,81.158,65.99%,80.74%,53.20%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629,81.183,65.68%,81.32%,53.51%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.290,80.934,66.75%,80.86%,53.53%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072,80.966,66.06%,81.22%,53.54%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [6]:
#group schools by schoolNameOverview
schoolNameOverview = completeDataDF.groupby("school_name")
schoolName = schoolNameOverview.count().index

#get the mean for each grade by school
Avg9 = completeDataDF.loc[completeDataDF["grade"] == "9th", :]
Avg9School = Avg9.groupby("school_name")
mathAvg9 = Avg9School["math_score"].mean()

Avg10 = completeDataDF.loc[completeDataDF["grade"] == "10th", :]
Avg10School = Avg10.groupby("school_name")
mathAvg10 = Avg10School["math_score"].mean()

Avg11 = completeDataDF.loc[completeDataDF["grade"] == "11th", :]
Avg11School = Avg11.groupby("school_name")
mathAvg11 = Avg11School["math_score"].mean()

Avg12 = completeDataDF.loc[completeDataDF["grade"] == "12th", :]
Avg12School = Avg12.groupby("school_name")
mathAvg12 = Avg12School["math_score"].mean()

#Math Average dataframe
mathGradeAvg = pd.DataFrame({"School Name": schoolName, "9th": mathAvg9, "10th": mathAvg10, 
                             "11th": mathAvg11, "12th": mathAvg12})

#format
mathGradeAvg["9th"] = mathGradeAvg["9th"].map("{:,.3f}".format)
mathGradeAvg["10th"] = mathGradeAvg["10th"].map("{:,.3f}".format)
mathGradeAvg["11th"] = mathGradeAvg["11th"].map("{:,.3f}".format)
mathGradeAvg["12th"] = mathGradeAvg["12th"].map("{:,.3f}".format)

#cleanup
mathGradeAvg = mathGradeAvg.reset_index(drop=True)
                           
mathGradeAvg.style.hide_index()

School Name,9th,10th,11th,12th
Bailey High School,77.084,76.997,77.516,76.492
Cabrera High School,83.095,83.155,82.766,83.277
Figueroa High School,76.403,76.540,76.884,77.151
Ford High School,77.361,77.672,76.918,76.180
Griffin High School,82.044,84.229,83.842,83.356
Hernandez High School,77.438,77.337,77.136,77.187
Holden High School,83.787,83.430,85.000,82.855
Huang High School,77.027,75.909,76.447,77.226
Johnson High School,77.188,76.691,77.492,76.863
Pena High School,83.625,83.372,84.328,84.122


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [7]:
#group by SchoolNameOverview
schoolNameOverview = completeDataDF.groupby("school_name")
schoolName = schoolNameOverview.count().index

#same thing as above
readingAvg9 = Avg9School["reading_score"].mean()
readingAvg10 = Avg10School["reading_score"].mean()
readingAvg11 = Avg11School["reading_score"].mean()
readingAvg12 = Avg12School["reading_score"].mean()

readingGradeAvg = pd.DataFrame({"School Name": schoolName, "9th": readingAvg9, "10th": readingAvg10, 
                             "11th": readingAvg11, "12th": readingAvg12})

#format
readingGradeAvg["9th"] = readingGradeAvg["9th"].map("{:,.3f}".format)
readingGradeAvg["10th"] = readingGradeAvg["10th"].map("{:,.3f}".format)
readingGradeAvg["11th"] = readingGradeAvg["11th"].map("{:,.3f}".format)
readingGradeAvg["12th"] = readingGradeAvg["12th"].map("{:,.3f}".format)

#cleanup
readingGradeAvg = readingGradeAvg.reset_index(drop=True)
                           
readingGradeAvg.style.hide_index()

School Name,9th,10th,11th,12th
Bailey High School,81.303,80.907,80.946,80.912
Cabrera High School,83.676,84.253,83.788,84.288
Figueroa High School,81.199,81.409,80.640,81.385
Ford High School,80.633,81.263,80.404,80.662
Griffin High School,83.369,83.707,84.288,84.014
Hernandez High School,80.867,80.660,81.396,80.857
Holden High School,83.677,83.325,83.816,84.699
Huang High School,81.290,81.512,81.417,80.306
Johnson High School,81.261,80.773,80.616,81.228
Pena High School,83.807,83.612,84.336,84.591


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

## Scores by School Size

* Perform the same operations as above, based on school size.

## Scores by School Type

* Perform the same operations as above, based on school type